In [1]:
# My Google Drive Mount하기!
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Latent Dirichlet Allocation (LDA)

**Week 07 page 1~17 정독하기!**

# gensim을 이용한 LDA Practice

# 1. TDM을 Corpus 형식으로 변환하기

In [2]:
import pandas as pd

df = pd.read_csv('https://github.com/euphoris/datasets/raw/master/neurips.zip')
df.head()

,year,title,abstract
0,2007,Competition Adds Complexity,It is known that determinining whether a DEC-P...
1,2007,Efficient Principled Learning of Thin Junction...,We present the first truly polynomial algorith...
2,2007,Regularized Boost for Semi-Supervised Learning,Semi-supervised inductive learning concerns ho...
3,2007,Simplified Rules and Theoretical Analysis for ...,We show that under suitable assumptions (prima...
4,2007,Predicting human gaze using low-level saliency...,"Under natural viewing conditions, human observ..."


In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

cv = TfidfVectorizer(
    max_features=2000,
    stop_words='english'
)

tdm = cv.fit_transform(df['abstract'])
print(tdm.shape)

(3920, 2000)


In [4]:
Term_list = cv.get_feature_names()
print(Term_list[:10])
print('# of Terms:', len(Term_list))

['000', '10', '100', '20', '2d', '3d', 'ability', 'able', 'absolute', 'abstract']
# of Terms: 2000


In [5]:
from gensim.matutils import Sparse2Corpus

corpus = Sparse2Corpus(tdm.T)

corpus[0]

[(1209, 0.19719051171057086),
 (1250, 0.1878641742740461),
 (265, 0.11655716303382495),
 (495, 0.21779310927686704),
 (316, 0.16096846113531893),
 (321, 0.25275451313424946),
 (317, 0.3778227141887415),
 (1573, 0.35429796717697476),
 (650, 0.32556474378711336),
 (1346, 0.3395230603787923),
 (1759, 0.32398854042493375),
 (767, 0.18302469761238765),
 (1756, 0.1205808084872785),
 (1219, 0.2021297737105109),
 (1282, 0.1848866146903614),
 (984, 0.2396176930939823)]

In [6]:
Idx2Term_dict = dict(enumerate(Term_list))
print(Idx2Term_dict[10])
print(Idx2Term_dict[20])
print(Idx2Term_dict[30])
print(Idx2Term_dict[40])
print(Idx2Term_dict[50])

accelerated
achieves
actually
additive
adversary


# 2. Corpus 형식으로 바로 변환하기

In [7]:
import re
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

In [8]:
GetRegularTokens = re.compile(
    r'\b\w{2,}\b',
    re.UNICODE
)
# 2글자 이상의 word를 모두 추출
# UNICODE를 사용함으로써, 영어, 한국어 등 다른 언어들에도 Applicable!

def GetListOfTokens(doc):

    Term_list = []
    doc = doc.lower()

    for token in GetRegularTokens.findall(doc):
        Term_list.append(token)

    return Term_list

In [9]:
print(GetListOfTokens(df['abstract'][0]))

['it', 'is', 'known', 'that', 'determinining', 'whether', 'dec', 'pomdp', 'namely', 'cooperative', 'partially', 'observable', 'stochastic', 'game', 'posg', 'has', 'cooperative', 'strategy', 'with', 'positive', 'expected', 'reward', 'is', 'complete', 'for', 'nexp', 'it', 'was', 'not', 'known', 'until', 'now', 'how', 'cooperation', 'affected', 'that', 'complexity', 'we', 'show', 'that', 'for', 'competitive', 'posgs', 'the', 'complexity', 'of', 'determining', 'whether', 'one', 'team', 'has', 'positive', 'expected', 'reward', 'strategy', 'is', 'complete', 'for', 'the', 'class', 'nexp', 'with', 'an', 'oracle', 'for', 'np']


In [10]:
totalword_list = []

DocIdxToTermList_list = []

for doc in df['abstract']:
    Term_list = GetListOfTokens(doc)
    DocIdxToTermList_list.append(Term_list)

In [11]:
len(DocIdxToTermList_list)

3920

In [15]:
from gensim.corpora.dictionary import Dictionary

IdxToTerm_dic = Dictionary(DocIdxToTermList_list)

In [16]:
# no_below: 해당 숫자 미만의 횟수만 나타나는 단어는 제외함 (너무 특수한 경우에만 나타남)
# no_above: 해당 percentage 초과한 문서에 나타나는 단어는 제외함 (너무 흔하게 나타남)

IdxToTerm_dic.filter_extremes(
    no_below=10,
    no_above=0.9
)

for i in range(10):
    print(IdxToTerm_dic[i])

affected
an
class
competitive
complete
complexity
cooperative
determining
expected
for


In [17]:
print(len(DocIdxToTermList_list))
print(len(IdxToTerm_dic))

3920
3516


In [18]:
DocIdxToCorpus_list = []

for doc in DocIdxToTermList_list:
    bow = IdxToTerm_dic.doc2bow(doc)
    DocIdxToCorpus_list.append(bow)

DocIdxToCorpus_list[100]

[(9, 3),
 (12, 1),
 (13, 1),
 (27, 1),
 (30, 1),
 (35, 1),
 (37, 1),
 (41, 1),
 (45, 2),
 (46, 1),
 (54, 2),
 (73, 1),
 (80, 1),
 (84, 1),
 (85, 2),
 (132, 1),
 (176, 2),
 (179, 3),
 (188, 1),
 (207, 1),
 (210, 1),
 (215, 1),
 (217, 1),
 (220, 1),
 (274, 1),
 (278, 1),
 (290, 1),
 (306, 1),
 (313, 1),
 (399, 1),
 (417, 3),
 (438, 1),
 (453, 1),
 (478, 1),
 (565, 1),
 (570, 1),
 (631, 1),
 (651, 1),
 (701, 1),
 (750, 1),
 (813, 1),
 (858, 1),
 (933, 1),
 (1028, 1),
 (1035, 1),
 (1040, 1),
 (1144, 1),
 (1200, 1),
 (1207, 1),
 (1309, 1),
 (1400, 1),
 (1502, 1),
 (1536, 1),
 (1538, 1),
 (1603, 1),
 (1963, 1),
 (2014, 1),
 (2015, 1),
 (2016, 1),
 (2017, 1)]

In [19]:
print(len(DocIdxToTermList_list))
print(len(DocIdxToCorpus_list))

3920
3920


# 3. LDA

In [20]:
from gensim.models.ldamodel import LdaModel
from sklearn.model_selection import train_test_split

In [21]:
train_corpus, val_corpus = \
    train_test_split(
        DocIdxToCorpus_list,
        test_size=0.2,
        random_state=1234
    )

In [24]:
print(len(train_corpus))
print(len(val_corpus))

3136
784


In [25]:
# DeprecationWarning 형태의 warning만 모두 ignore할 수 있도록 세팅해주자. (Wow... bam...)
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [27]:
model = LdaModel(
    corpus=train_corpus,
    num_topics=100,
    id2word=IdxToTerm_dic,
    iterations=50,
    random_state=1234
)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [28]:
loss_val = model.log_perplexity(val_corpus)
print(loss_val)

-12.355450172416207


* loss_val이 negative value이므로, 반복 update함으로써, loss를 줄여주자.
* (개선 폭이 0.1보다 작아지면 아래의 loop을 중단!)

In [30]:
import numpy as np

old_loss = -np.inf
loss = loss_val

while loss - old_loss > 0.1:
    model.update(train_corpus)
    old_loss = loss
    loss = model.log_perplexity(val_corpus)
    print(loss)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


-11.241231622575837
-10.720267282143102
-10.465074214353871
-10.329508765830878
-10.244000699610675


In [31]:
FolderPath = '/content/drive/MyDrive/03. Kookmin AI Big Data MBA/Semester 3_032021-062021/1. Text Data Analytics/Lecture Notes Review/data'

model.save(FolderPath + '/' + 'lda-model')

* lda-model 이름을 가진 여러 개의 files가 생성되는데, 너무 복잡하므로, 아래와 같이 zipped file을 생성하자.
* data/lda-model로 시작하는 이름을 가진 모든 파일들을, data/mylda.zip 이름으로 zipped file 만들어라!

In [35]:
# ! zip data/mylda.zip data/lda-model*

Saved 된 model을 Load하는 방법!

In [36]:
# model = LdaModel.load('data/lda-model')

# 4. LDA 결과 보기

* 0번째 Topic에 자주 나오는 단어들 확인하기!
* 각 확률은 해당 Topic에서 각 단어가 나올 확률임!

In [39]:
model.show_topic(0)

[('gradient', 0.06881777),
 ('convergence', 0.044986233),
 ('stochastic', 0.04461693),
 ('optimization', 0.040169924),
 ('descent', 0.033516023),
 ('convex', 0.02372339),
 ('for', 0.020055752),
 ('method', 0.019394679),
 ('methods', 0.014865807),
 ('this', 0.013995724)]

In [40]:
model.show_topic(1)

[('conditional', 0.07255719),
 ('distributions', 0.04472041),
 ('exponential', 0.04347621),
 ('regression', 0.04069891),
 ('semi', 0.02837944),
 ('variables', 0.025541848),
 ('data', 0.024748307),
 ('supervised', 0.024272107),
 ('as', 0.023778958),
 ('count', 0.020175267)]

In [41]:
model.show_topic(2)

[('manifold', 0.03447574),
 ('method', 0.027862942),
 ('that', 0.02444804),
 ('is', 0.01681172),
 ('riemannian', 0.0166332),
 ('denoising', 0.016502617),
 ('on', 0.016303292),
 ('problem', 0.016148485),
 ('proposed', 0.013076085),
 ('for', 0.012481662)]

**algorithm**이라는 term과 관련된 Topic을 확인해보자!

(1) **algorithm**이라는 term이 **dic**에서 몇 번째 index에 있는지 확인해보자.

In [42]:
IdxToTerm_dic.token2id['algorithm']

35

(2) **algorithm**이라는 term이 나타날 확률이 3% 이상인 Topic들이 어떤 것들이 있는지 찾아보자.

In [49]:
model.get_term_topics(35, minimum_probability=0.03)

[(52, 0.039220776)]

(3) 해당 52번째 Topic에 어떤 term들이 있는지 확인해보자!

In [50]:
model.show_topic(52)

[('is', 0.04574188),
 ('algorithm', 0.039241396),
 ('bound', 0.026241753),
 ('an', 0.023322212),
 ('that', 0.021194356),
 ('for', 0.018767951),
 ('it', 0.017022893),
 ('learning', 0.015932122),
 ('this', 0.015253211),
 ('active', 0.0122775305)]

아래 **new_text** 사례는 어떤 Topic으로 분류가 되는지 한 번 알아볼까나~?

In [51]:
# 여러 line들롤 이루어진 text를 쓸 때는 quotation mark를 3개를 입력한다! (wow!)
NewDoc = '''We describe latent Dirichlet allocation (LDA), a generative probabilistic model for collections of
discrete data such as text corpora. LDA is a three-level hierarchical Bayesian model, in which each
item of a collection is modeled as a finite mixture over an underlying set of topics. Each topic is, in
turn, modeled as an infinite mixture over an underlying set of topic probabilities. In the context of
text modeling, the topic probabilities provide an explicit representation of a document. We present
efficient approximate inference techniques based on variational methods and an EM algorithm for
empirical Bayes parameter estimation. We report results in document modeling, text classification,
and collaborative filtering, comparing to a mixture of unigrams model and the probabilistic LSI
model.'''

In [52]:
doc = GetListOfTokens(NewDoc)
bow = IdxToTerm_dic.doc2bow(doc)
bow

[(1, 5),
 (9, 2),
 (13, 3),
 (35, 1),
 (56, 1),
 (65, 1),
 (78, 1),
 (84, 1),
 (85, 2),
 (90, 1),
 (109, 2),
 (118, 1),
 (126, 1),
 (155, 1),
 (158, 2),
 (179, 3),
 (239, 1),
 (268, 1),
 (274, 4),
 (283, 2),
 (291, 1),
 (306, 1),
 (313, 1),
 (343, 2),
 (376, 3),
 (381, 3),
 (382, 1),
 (399, 1),
 (410, 1),
 (456, 1),
 (462, 1),
 (475, 2),
 (521, 1),
 (530, 2),
 (536, 1),
 (570, 1),
 (617, 1),
 (636, 1),
 (646, 2),
 (745, 1),
 (754, 1),
 (834, 1),
 (884, 1),
 (901, 1),
 (925, 1),
 (933, 1),
 (988, 1),
 (1013, 1),
 (1018, 1),
 (1019, 1),
 (1077, 1),
 (1080, 3),
 (1131, 1),
 (1309, 1),
 (1447, 2),
 (1466, 1),
 (1502, 1),
 (1660, 1),
 (1741, 2),
 (1883, 1),
 (2071, 1),
 (2211, 1),
 (2425, 1),
 (2909, 1)]

In [59]:
model.get_document_topics(bow)

[(8, 0.072855406),
 (9, 0.1426797),
 (16, 0.078463726),
 (45, 0.021703325),
 (51, 0.05348625),
 (53, 0.048506808),
 (65, 0.26158482),
 (74, 0.268745),
 (94, 0.04219001)]

In [60]:
model.show_topic(74)

[('topic', 0.04125076),
 ('documents', 0.028033545),
 ('document', 0.026193501),
 ('model', 0.02564058),
 ('text', 0.025590831),
 ('that', 0.025432939),
 ('with', 0.020255445),
 ('words', 0.018574415),
 ('is', 0.018556202),
 ('data', 0.017897585)]

# 5. LDAvis를 통한 결과 Visualization

In [61]:
! pip install pyLDAvis==2.1.2

     |████████████████████████████████| 1.6 MB 12.3 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97738 sha256=353bf032698882544b375196bc3c9f0007275ec328e37b0e034a189922a86f4f
  Stored in directory: /root/.cache/pip/wheels/3b/fb/41/e32e5312da9f440d34c4eff0d2207b46dc9332a7b931ef1e89
Successfully built pyLDAvis


In [62]:
import pyLDAvis.gensim

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


아래의 code는, jupyter notebook에 applicable하고자 함!

In [63]:
pyLDAvis.enable_notebook()

In [64]:
p = pyLDAvis.gensim.prepare(
    topic_model=model,
    corpus=DocIdxToCorpus_list,
    dictionary=IdxToTerm_dic
)

In [65]:
pyLDAvis.display(p)

* 각 Topic의 Index가 이 Vis에서는 1번부터 시작함에 주의!

# 6. 현저성과 관련성 (Saliency & Relevance)

**Week 07 page 19-23 정독하기**

# LDA Evaluations

# 1. 혼란도 (Perplexity)

* Perplexity의 의미
    * "이런 결과가 나올 리가 없는데?" : Perplexity 높음
    * "그래.. 이런 결과가 나올 법하지..": Perplexity 낮음

**Week 07 page 24-27 정독하기**

# 2. 주제 응집도와 다양도 (Topic Coherence & Diversity)

**Week 07 page 28-34 정독하기**

# 3. 주제 응집도와 다양도 계산 (Topic Coherence & Diversity)

In [66]:
from gensim.models import CoherenceModel

In [70]:
coh = CoherenceModel(
    model=model,
    corpus=DocIdxToCorpus_list,
    texts=DocIdxToTermList_list,
    dictionary=IdxToTerm_dic,
    coherence='c_v'
)

In [71]:
coh.get_coherence()

0.2947702188423044

In [72]:
Topic_cnt = 25
Topic_set = set() # 중복된 element를 제거하는 집합 형태임!

for TopicIdx in range(model.num_topics):
    for term, prob in model.show_topic(TopicIdx):
        Topic_set.add(term)

In [73]:
print(Topic_set)

{'mixture', 'alternatives', 'marginal', 'programming', 'means', 'calibration', 'search', 'submodular', 'physical', 'regression', 'online', 'motion', 'entropy', 'errors', 'theta', 'submodularity', 'on', 'pca', 'feature', 'music', 'group', 'with', 'different', 'number', 'trees', 'topic', 'vector', 'gaussian', 'making', 'generalization', 'neural', 'structure', 'behavioral', 'temporal', 'generative', 'convex', 'when', 'case', 'recognition', 'bregman', 'planning', 'alignment', 'dictionary', 'exploration', 'more', 'category', 'goals', 'cover', 'emph', 'visual', 'lda', 'gradient', 'policies', 'face', 'proposal', 'mechanism', 'place', 'filters', 'spike', 'component', 'nearest', 'be', 'semidefinite', 'ranking', 'posterior', 'fast', 'algorithm', 'beliefs', 'active', 'tasks', 'across', 'modeling', 'space', 'partially', 'backward', 'supervised', 'inference', 'riemannian', 'word', 'linear', 'networks', 'coding', 'kernel', 'al', 'graph', 'abstract', 'domain', 'dynamic', 'this', 'items', 'bounds', 'p

In [74]:
print(len(Topic_set))

419


In [75]:
print(len(Topic_set) / (model.num_topics * Topic_cnt))

0.1676


따라서, 주제의 다양도는 43% 정도로, 약한 편이구나!